In [125]:
# This is a very simple implementation of the UCT Monte Carlo Tree Search algorithm in Python 2.7.
# The function UCT(rootstate, itermax, verbose = False) is towards the bottom of the code.
# It aims to have the clearest and simplest possible code, and for the sake of clarity, the code
# is orders of magnitude less efficient than it could be made, particularly by using a 
# state.GetRandomMove() or state.DoRandomRollout() function.
# 
# Example GameState classes for Nim, OXO and Othello are included to give some idea of how you
# can write your own GameState use UCT in your 2-player game. Change the game to be played in 
# the UCTPlayGame() function at the bottom of the code.
# 
# Written by Peter Cowling, Ed Powley, Daniel Whitehouse (University of York, UK) September 2012.
# 
# Licence is granted to freely use and distribute for any sensible/legal purpose so long as this comment
# remains in any distributed code.
# 
# For more information about Monte Carlo Tree Search check out our web site at www.mcts.ai

from math import *
import random
import numpy as np
import pandas as pd

class GameState:
    """ A state of the game, i.e. the game board. These are the only functions which are
        absolutely necessary to implement UCT in any 2-player complete information deterministic 
        zero-sum game, although they can be enhanced and made quicker, for example by using a 
        GetRandomMove() function to generate a random move during rollout.
        By convention the players are numbered 1 and 2.
    """
    def __init__(self):
            self.playerJustMoved = 2 # At the root pretend the player just moved is player 2 - player 1 has the first move
        
    def Clone(self):
        """ Create a deep clone of this game state.
        """
        st = GameState()
        st.playerJustMoved = self.playerJustMoved
        return st

    def DoMove(self, move):
        """ Update a state by carrying out the given move.
            Must update playerJustMoved.
        """
        self.playerJustMoved = 3 - self.playerJustMoved
        
    def GetMoves(self):
        """ Get all possible moves from this state.
        """
    
    def GetResult(self, playerjm):
        """ Get the game result from the viewpoint of playerjm. 
        """

    def __repr__(self):
        """ Don't need this - but good style.
        """
        pass


class OXOState:
    """ A state of the game, i.e. the game board.
        Squares in the board are in this arrangement
        012
        345
        678
        where 0 = empty, 1 = player 1 (X), 2 = player 2 (O)
    """
    def __init__(self):
        self.playerJustMoved = 2 # At the root pretend the player just moved is p2 - p1 has the first move
        self.board = [0,0,0,0,0,0,0,0,0] # 0 = empty, 1 = player 1, 2 = player 2
        
    def Clone(self):
        """ Create a deep clone of this game state.
        """
        st = OXOState()
        st.playerJustMoved = self.playerJustMoved
        st.board = self.board[:]
        return st

    def DoMove(self, move):
        """ Update a state by carrying out the given move.
            Must update playerToMove.
        """
        assert move >= 0 and move <= 8 and move == int(move) and self.board[move] == 0
        self.playerJustMoved = 3 - self.playerJustMoved
        self.board[move] = self.playerJustMoved
        
    def GetMoves(self):
        """ Get all possible moves from this state.
        """
        return [i for i in range(9) if self.board[i] == 0]
    
    def GetResult(self, playerjm):
        """ Get the game result from the viewpoint of playerjm. 
        """
        for (x,y,z) in [(0,1,2),(3,4,5),(6,7,8),(0,3,6),(1,4,7),(2,5,8),(0,4,8),(2,4,6)]:
            if self.board[x] == self.board[y] == self.board[z]:
                if self.board[x] == playerjm:
                    return 1.0
                else:
                    return 0.0
        if self.GetMoves() == []: return 0.5 # draw
        return False # Should not be possible to get here

    def __repr__(self):
        s= ""
        for i in range(9): 
            s += ".XO"[self.board[i]]
            if i % 3 == 2: s += "\n"
        return s


class Node:
    """ A node in the game tree. Note wins is always from the viewpoint of playerJustMoved.
        Crashes if state not specified.
    """
    def __init__(self, move = None, parent = None, state = None):
        self.move = move # the move that got us to this node - "None" for the root node
        self.parentNode = parent # "None" for the root node
        self.childNodes = []
        self.wins = 0
        self.visits = 0
        self.untriedMoves = state.GetMoves() # future child nodes
        self.playerJustMoved = state.playerJustMoved # the only part of the state that the Node needs later
        
    def UCTSelectChild(self):
        """ Use the UCB1 formula to select a child node. Often a constant UCTK is applied so we have
            lambda c: c.wins/c.visits + UCTK * sqrt(2*log(self.visits)/c.visits to vary the amount of
            exploration versus exploitation.
        """
        s = sorted(self.childNodes, key = lambda c: c.wins/c.visits + sqrt(2*log(self.visits)/c.visits))[-1]
        return s
    
    def AddChild(self, m, s):
        """ Remove m from untriedMoves and add a new child node for this move.
            Return the added child node
        """
        n = Node(move = m, parent = self, state = s)
        self.untriedMoves.remove(m)
        self.childNodes.append(n)
        return n
    
    def Update(self, result):
        """ Update this node - one additional visit and result additional wins. result must be from the viewpoint of playerJustmoved.
        """
        self.visits += 1
        self.wins += result

    def __repr__(self):
        return "[M:" + str(self.move) + " W/V:" + str(self.wins) + "/" + str(self.visits) + " U:" + str(self.untriedMoves) + "]"

    def TreeToString(self, indent):
        s = self.IndentString(indent) + str(self)
        for c in self.childNodes:
             s += c.TreeToString(indent+1)
        return s

    def IndentString(self,indent):
        s = "\n"
        for i in range (1,indent+1):
            s += "| "
        return s

    def ChildrenToString(self):
        s = ""
        for c in self.childNodes:
             s += str(c) + "\n"
        return s


def UCT(rootstate, itermax, verbose = False):
    """ Conduct a UCT search for itermax iterations starting from rootstate.
        Return the best move from the rootstate.
        Assumes 2 alternating players (player 1 starts), with game results in the range [0.0, 1.0]."""

    
    rootnode = Node(state = rootstate)

    for i in range(itermax):
        node = rootnode
        state = rootstate.Clone()

        # Select
        while node.untriedMoves == [] and node.childNodes != []: # node is fully expanded and non-terminal
            node = node.UCTSelectChild()
            state.DoMove(node.move)

        # Expand
        if node.untriedMoves != []:  # if we can expand (i.e. state/node is non-terminal)
            m = random.choice(node.untriedMoves) 
            state.DoMove(m)
            node = node.AddChild(m, state)  # add child and descend tree

        # Rollout - this can often be made orders of magnitude quicker using a state.GetRandomMove() function
        while state.GetMoves() != []: # while state is non-terminal
            state.DoMove(random.choice(state.GetMoves()))

        # Backpropagate
        while node != None: # backpropagate from the expanded node and work back to the root node
            node.Update(state.GetResult(node.playerJustMoved)) # state is terminal. Update node with result from POV of node.playerJustMoved
            node = node.parentNode

    # Output some information about the tree - can be omitted
    if verbose: print(rootnode.TreeToString(0))
    else: print(rootnode.ChildrenToString())

    return sorted(rootnode.childNodes, key = lambda c: c.visits)[-1].move # return the move that was most visited
                
def UCTPlayGame():
    """ Play a sample game between two UCT players where each player gets a different number 
        of UCT iterations (= simulations = tree nodes).
    """
    state_n_moveP1=np.array([],dtype=int)
    state_n_moveP2=np.array([],dtype=int)
    num_of_movesP1=0
    num_of_movesP2=0
    
    state = OXOState() # uncomment to play OXO
    while state.GetMoves() != []:
        print(str(state))
        if state.playerJustMoved == 1:
            m = UCT(rootstate=state, itermax=1000, verbose=False)  # play with values for itermax and verbose = True
        else:
            m = UCT(rootstate=state, itermax=100, verbose=False)
        print("Best Move: " + str(m) + "\n")
        
        temp=np.hstack((state.playerJustMoved,state.board,m))
        if state.playerJustMoved == 1:
            state_n_moveP1=np.append(state_n_moveP1, temp, axis=0)
            num_of_movesP1+=1
        else:
            state_n_moveP2=np.append(state_n_moveP2, temp, axis=0)
            num_of_movesP2+=1
        
        state.DoMove(m)
        if state.GetResult(state.playerJustMoved) != False:
            print(str(state))
            break
    
    state_n_moveP1=state_n_moveP1.reshape((num_of_movesP1,11))
    state_n_moveP2=state_n_moveP2.reshape((num_of_movesP2,11))
    print("P1 all states: \n",state_n_moveP1,"\n")
    print("P2 all states: \n",state_n_moveP2,"\n")
    
    if state.GetResult(state.playerJustMoved) == 1.0:
        Winner=state.playerJustMoved #Playerx wins
        print("Player " + str(state.playerJustMoved) + " wins!")
    elif state.GetResult(state.playerJustMoved) == 0.0:
        Winner=3 -state.playerJustMoved #Player other than x wins
        print("Player " + str(3 - state.playerJustMoved) + " wins!")
    else:
        Winner=0 #Match draw
        print("Nobody wins!")
    
    if Winner==1:
        best_moves=state_n_moveP1
    elif Winner==2:
        best_moves=state_n_moveP2
    else:
        best_moves=np.append(state_n_moveP1,state_n_moveP2, axis=0)
    return best_moves

if __name__ == "__main__":
    """ Play a single game to the end using UCT for both players. 
    """
    Best_moves=np.zeros((1,11),dtype=object)
    Indexes=np.array(['Player','s[0]','s[1]','s[2]','s[3]','s[4]','s[5]','s[6]','s[7]','s[8]','Move'],dtype=object)
    for i in range (0,500):
        best_moves=UCTPlayGame()
        Best_moves=np.append(Best_moves, best_moves, axis=0)
    Best_moves[0,:]=Indexes
    print("Best moves are: \n",Best_moves)
    
    df = pd.DataFrame(Best_moves)
    print (df)
    df.to_csv('Task1_bestmoves.csv', sep=',', index=False, index_label=False)

...
...
...

[M:7 W/V:3.5/7 U:[0, 2]]
[M:1 W/V:3.5/7 U:[5, 6]]
[M:0 W/V:10.5/13 U:[]]
[M:4 W/V:10.5/13 U:[]]
[M:2 W/V:8.0/11 U:[]]
[M:6 W/V:7.0/10 U:[]]
[M:5 W/V:13.0/15 U:[]]
[M:3 W/V:12.0/15 U:[]]
[M:8 W/V:5.0/9 U:[]]

Best Move: 3

...
X..
...

[M:4 W/V:123.5/280 U:[]]
[M:6 W/V:19.5/79 U:[]]
[M:7 W/V:9.5/57 U:[]]
[M:5 W/V:6.5/49 U:[]]
[M:1 W/V:29.5/101 U:[]]
[M:0 W/V:103.0/242 U:[]]
[M:8 W/V:12.5/64 U:[]]
[M:2 W/V:43.0/128 U:[]]

Best Move: 4

...
XO.
...

[M:1 W/V:8.0/15 U:[]]
[M:0 W/V:11.5/19 U:[]]
[M:6 W/V:6.0/13 U:[]]
[M:5 W/V:0.0/6 U:[8]]
[M:8 W/V:7.5/14 U:[]]
[M:7 W/V:2.5/9 U:[]]
[M:2 W/V:17.5/24 U:[]]

Best Move: 2

..X
XO.
...

[M:5 W/V:6.0/42 U:[]]
[M:6 W/V:28.5/88 U:[]]
[M:7 W/V:47.0/122 U:[]]
[M:1 W/V:46.5/121 U:[]]
[M:8 W/V:31.5/93 U:[]]
[M:0 W/V:302.0/534 U:[]]

Best Move: 0

O.X
XO.
...

[M:6 W/V:4.0/14 U:[]]
[M:1 W/V:3.0/13 U:[]]
[M:7 W/V:5.0/15 U:[]]
[M:5 W/V:2.0/11 U:[]]
[M:8 W/V:31.5/47 U:[]]

Best Move: 8

O.X
XO.
..X

[M:7 W/V:326.0/621 U:[]]
[M:5 W/V:121.5/271 U

[M:3 W/V:49.0/129 U:[]]
[M:1 W/V:255.5/472 U:[]]
[M:5 W/V:59.5/148 U:[]]
[M:7 W/V:28.5/90 U:[]]
[M:0 W/V:28.5/90 U:[]]
[M:6 W/V:19.0/71 U:[]]

Best Move: 1

.OX
.O.
..X

[M:3 W/V:4.5/13 U:[]]
[M:5 W/V:6.0/16 U:[]]
[M:0 W/V:9.0/20 U:[]]
[M:6 W/V:0.0/7 U:[]]
[M:7 W/V:31.0/44 U:[]]

Best Move: 7

.OX
.O.
.XX

[M:5 W/V:445.0/852 U:[]]
[M:3 W/V:10.0/60 U:[]]
[M:6 W/V:8.0/55 U:[]]
[M:0 W/V:0.0/33 U:[]]

Best Move: 5

.OX
.OO
.XX

[M:6 W/V:3.0/16 U:[]]
[M:3 W/V:40.5/69 U:[]]
[M:0 W/V:2.5/15 U:[]]

Best Move: 3

.OX
XOO
.XX

[M:0 W/V:0.0/36 U:[]]
[M:6 W/V:482.0/964 U:[]]

Best Move: 6

.OX
XOO
OXX

[M:0 W/V:50.0/100 U:[]]

Best Move: 0

XOX
XOO
OXX

P1 all states: 
 [[1 0 0 1 0 0 0 0 0 0 4]
 [1 0 0 1 0 2 0 0 0 1 1]
 [1 0 2 1 0 2 0 0 1 1 5]
 [1 0 2 1 1 2 2 0 1 1 6]] 

P2 all states: 
 [[2 0 0 0 0 0 0 0 0 0 2]
 [2 0 0 1 0 2 0 0 0 0 8]
 [2 0 2 1 0 2 0 0 0 1 7]
 [2 0 2 1 0 2 2 0 1 1 3]
 [2 0 2 1 1 2 2 2 1 1 0]] 

Nobody wins!
...
...
...

[M:4 W/V:14.0/15 U:[]]
[M:7 W/V:14.0/15 U:[]]
[M:1 W/V:6.5/


Best Move: 4

...
.X.
...

[M:1 W/V:31.0/131 U:[]]
[M:0 W/V:120.0/333 U:[]]
[M:3 W/V:19.0/100 U:[]]
[M:6 W/V:23.0/110 U:[]]
[M:8 W/V:7.0/66 U:[]]
[M:5 W/V:7.0/66 U:[]]
[M:7 W/V:9.0/72 U:[]]
[M:2 W/V:27.5/122 U:[]]

Best Move: 0

O..
.X.
...

[M:5 W/V:11.0/16 U:[]]
[M:2 W/V:12.0/17 U:[]]
[M:6 W/V:8.0/13 U:[]]
[M:8 W/V:6.0/12 U:[]]
[M:3 W/V:9.5/15 U:[]]
[M:7 W/V:4.0/9 U:[]]
[M:1 W/V:12.5/18 U:[]]

Best Move: 1

OX.
.X.
...

[M:2 W/V:23.5/124 U:[]]
[M:6 W/V:59.0/218 U:[]]
[M:5 W/V:19.5/113 U:[]]
[M:7 W/V:74.5/256 U:[]]
[M:8 W/V:9.0/79 U:[]]
[M:3 W/V:56.0/210 U:[]]

Best Move: 7

OX.
.X.
.O.

[M:8 W/V:9.0/15 U:[]]
[M:5 W/V:8.5/15 U:[]]
[M:3 W/V:21.0/27 U:[]]
[M:2 W/V:10.0/16 U:[]]
[M:6 W/V:21.5/27 U:[]]

Best Move: 6

OX.
.X.
XO.

[M:2 W/V:391.5/831 U:[]]
[M:5 W/V:9.0/65 U:[]]
[M:8 W/V:0.0/39 U:[]]
[M:3 W/V:9.0/65 U:[]]

Best Move: 2

OXO
.X.
XO.

[M:3 W/V:23.0/37 U:[]]
[M:8 W/V:12.5/25 U:[]]
[M:5 W/V:23.5/38 U:[]]

Best Move: 5

OXO
.XX
XO.

[M:3 W/V:482.0/964 U:[]]
[M:8 W/V:0.0/36 U:[]]

[M:8 W/V:7.0/50 U:[]]
[M:7 W/V:7.0/50 U:[]]
[M:3 W/V:458.5/841 U:[]]
[M:5 W/V:11.0/59 U:[]]

Best Move: 3

XXO
OO.
X..

[M:8 W/V:2.0/17 U:[]]
[M:7 W/V:3.0/19 U:[]]
[M:5 W/V:32.0/64 U:[]]

Best Move: 5

XXO
OOX
X..

[M:7 W/V:250.0/500 U:[]]
[M:8 W/V:250.0/500 U:[]]

Best Move: 8

XXO
OOX
X.O

[M:7 W/V:50.0/100 U:[]]

Best Move: 7

XXO
OOX
XXO

P1 all states: 
 [[1 1 0 0 0 0 0 0 0 0 4]
 [1 1 1 0 0 2 0 0 0 0 2]
 [1 1 1 2 0 2 0 1 0 0 3]
 [1 1 1 2 2 2 1 1 0 0 8]] 

P2 all states: 
 [[2 0 0 0 0 0 0 0 0 0 0]
 [2 1 0 0 0 2 0 0 0 0 1]
 [2 1 1 2 0 2 0 0 0 0 6]
 [2 1 1 2 2 2 0 1 0 0 5]
 [2 1 1 2 2 2 1 1 0 2 7]] 

Nobody wins!
...
...
...

[M:0 W/V:9.0/13 U:[]]
[M:4 W/V:12.0/15 U:[]]
[M:1 W/V:7.5/11 U:[]]
[M:6 W/V:9.5/13 U:[]]
[M:7 W/V:7.0/11 U:[]]
[M:5 W/V:2.0/6 U:[6, 7, 8]]
[M:8 W/V:4.0/8 U:[1]]
[M:3 W/V:7.5/11 U:[]]
[M:2 W/V:8.5/12 U:[]]

Best Move: 4

...
.X.
...

[M:3 W/V:30.0/141 U:[]]
[M:2 W/V:33.0/149 U:[]]
[M:6 W/V:32.5/147 U:[]]
[M:8 W/V:5.0/68 U:[]]
[M:7 W/V:10.0/84 U:[]]
[M:0 W/V:57.5/

[M:7 W/V:12.0/54 U:[]]
[M:4 W/V:62.5/147 U:[]]
[M:2 W/V:54.0/132 U:[]]
[M:0 W/V:103.0/215 U:[]]
[M:1 W/V:60.5/144 U:[]]
[M:3 W/V:34.5/97 U:[]]
[M:6 W/V:71.0/162 U:[]]
[M:5 W/V:9.5/49 U:[]]

Best Move: 0

O..
...
..X

[M:4 W/V:10.5/17 U:[]]
[M:5 W/V:2.5/8 U:[]]
[M:6 W/V:12.0/18 U:[]]
[M:7 W/V:7.5/14 U:[]]
[M:2 W/V:8.0/14 U:[]]
[M:1 W/V:11.0/17 U:[]]
[M:3 W/V:5.5/12 U:[]]

Best Move: 6

O..
...
X.X

[M:3 W/V:15.5/66 U:[]]
[M:5 W/V:17.5/71 U:[]]
[M:7 W/V:90.5/208 U:[]]
[M:4 W/V:43.5/123 U:[]]
[M:2 W/V:199.0/392 U:[]]
[M:1 W/V:53.0/140 U:[]]

Best Move: 2

O.O
...
X.X

[M:3 W/V:1.5/10 U:[]]
[M:4 W/V:5.0/15 U:[]]
[M:7 W/V:6.0/16 U:[]]
[M:1 W/V:30.5/45 U:[]]
[M:5 W/V:4.5/14 U:[]]

Best Move: 1

OXO
...
X.X

[M:3 W/V:9.5/62 U:[]]
[M:5 W/V:8.0/58 U:[]]
[M:7 W/V:409.0/818 U:[]]
[M:4 W/V:9.5/62 U:[]]

Best Move: 7

OXO
...
XOX

[M:4 W/V:16.5/33 U:[]]
[M:3 W/V:16.5/33 U:[]]
[M:5 W/V:17.0/34 U:[]]

Best Move: 5

OXO
..X
XOX

[M:4 W/V:250.0/500 U:[]]
[M:3 W/V:250.0/500 U:[]]

Best Move: 3

OXO
O.X


...

P1 all states: 
 [[1 0 0 0 0 1 0 0 0 0 0]
 [1 2 0 0 1 1 0 0 0 0 5]
 [1 2 1 0 1 1 2 0 0 0 7]
 [1 2 1 1 1 1 2 0 2 0 6]] 

P2 all states: 
 [[2 0 0 0 0 0 0 0 0 0 4]
 [2 2 0 0 0 1 0 0 0 0 3]
 [2 2 0 0 1 1 2 0 0 0 1]
 [2 2 1 0 1 1 2 0 2 0 2]
 [2 2 1 1 1 1 2 2 2 0 8]] 

Nobody wins!
...
...
...

[M:3 W/V:10.5/13 U:[]]
[M:4 W/V:10.0/13 U:[]]
[M:8 W/V:3.5/7 U:[4, 6]]
[M:1 W/V:9.5/12 U:[]]
[M:6 W/V:5.0/8 U:[2]]
[M:0 W/V:12.0/14 U:[]]
[M:7 W/V:7.0/10 U:[]]
[M:2 W/V:14.0/16 U:[]]
[M:5 W/V:3.0/7 U:[3, 7]]

Best Move: 2

..X
...
...

[M:4 W/V:74.0/202 U:[]]
[M:6 W/V:39.0/129 U:[]]
[M:0 W/V:100.0/253 U:[]]
[M:3 W/V:33.5/117 U:[]]
[M:5 W/V:17.5/81 U:[]]
[M:1 W/V:10.0/63 U:[]]
[M:8 W/V:15.5/77 U:[]]
[M:7 W/V:16.0/78 U:[]]

Best Move: 0

O.X
...
...

[M:6 W/V:9.0/15 U:[]]
[M:5 W/V:7.0/13 U:[]]
[M:4 W/V:14.5/21 U:[]]
[M:7 W/V:3.5/9 U:[]]
[M:3 W/V:10.0/16 U:[]]
[M:8 W/V:8.0/14 U:[]]
[M:1 W/V:5.5/12 U:[]]

Best Move: 4

O.X
.X.
...

[M:5 W/V:14.5/79 U:[]]
[M:8 W/V:5.0/54 U:[]]
[M:1 W/V:8.0/62 U:[]]
[M:7 W

[M:3 W/V:24.0/102 U:[]]
[M:1 W/V:9.5/66 U:[]]
[M:7 W/V:9.5/66 U:[]]
[M:5 W/V:11.5/71 U:[]]
[M:6 W/V:295.5/642 U:[]]
[M:8 W/V:5.0/53 U:[]]

Best Move: 6

O.X
.X.
O..

[M:5 W/V:7.0/15 U:[]]
[M:3 W/V:41.5/50 U:[]]
[M:8 W/V:2.0/9 U:[]]
[M:1 W/V:4.5/12 U:[]]
[M:7 W/V:5.5/14 U:[]]

Best Move: 3

O.X
XX.
O..

[M:5 W/V:387.5/824 U:[]]
[M:7 W/V:10.5/69 U:[]]
[M:8 W/V:0.0/39 U:[]]
[M:1 W/V:10.0/68 U:[]]

Best Move: 5

O.X
XXO
O..

[M:7 W/V:21.5/36 U:[]]
[M:1 W/V:23.0/38 U:[]]
[M:8 W/V:13.0/26 U:[]]

Best Move: 1

OXX
XXO
O..

[M:7 W/V:482.0/964 U:[]]
[M:8 W/V:0.0/36 U:[]]

Best Move: 7

OXX
XXO
OO.

[M:8 W/V:50.0/100 U:[]]

Best Move: 8

OXX
XXO
OOX

P1 all states: 
 [[1 0 0 0 0 1 0 0 0 0 0]
 [1 2 0 1 0 1 0 0 0 0 6]
 [1 2 0 1 1 1 0 2 0 0 5]
 [1 2 1 1 1 1 2 2 0 0 7]] 

P2 all states: 
 [[2 0 0 0 0 0 0 0 0 0 4]
 [2 2 0 0 0 1 0 0 0 0 2]
 [2 2 0 1 0 1 0 2 0 0 3]
 [2 2 0 1 1 1 2 2 0 0 1]
 [2 2 1 1 1 1 2 2 2 0 8]] 

Nobody wins!
...
...
...

[M:0 W/V:12.5/15 U:[]]
[M:7 W/V:9.0/12 U:[]]
[M:5 W/V:6.5/10

[M:6 W/V:0.0/36 U:[]]
[M:8 W/V:482.0/964 U:[]]

Best Move: 8

XOX
OOX
.XO

[M:6 W/V:50.0/100 U:[]]

Best Move: 6

XOX
OOX
XXO

P1 all states: 
 [[1 0 0 1 0 0 0 0 0 0 4]
 [1 1 0 1 0 2 0 0 0 0 1]
 [1 1 2 1 0 2 0 0 1 0 3]
 [1 1 2 1 2 2 1 0 1 0 8]] 

P2 all states: 
 [[2 0 0 0 0 0 0 0 0 0 2]
 [2 0 0 1 0 2 0 0 0 0 0]
 [2 1 2 1 0 2 0 0 0 0 7]
 [2 1 2 1 2 2 0 0 1 0 5]
 [2 1 2 1 2 2 1 0 1 2 6]] 

Nobody wins!
...
...
...

[M:6 W/V:6.5/11 U:[]]
[M:4 W/V:13.0/16 U:[]]
[M:7 W/V:5.0/9 U:[]]
[M:1 W/V:8.5/12 U:[]]
[M:5 W/V:5.0/9 U:[]]
[M:2 W/V:8.5/12 U:[]]
[M:8 W/V:3.0/7 U:[0, 2]]
[M:0 W/V:12.0/15 U:[]]
[M:3 W/V:5.0/9 U:[]]

Best Move: 4

...
.X.
...

[M:3 W/V:34.5/150 U:[]]
[M:0 W/V:56.0/204 U:[]]
[M:8 W/V:7.0/73 U:[]]
[M:1 W/V:27.5/131 U:[]]
[M:6 W/V:20.0/111 U:[]]
[M:5 W/V:14.0/94 U:[]]
[M:2 W/V:35.5/152 U:[]]
[M:7 W/V:11.0/85 U:[]]

Best Move: 0

O..
.X.
...

[M:7 W/V:7.0/13 U:[]]
[M:6 W/V:5.5/11 U:[]]
[M:5 W/V:3.0/9 U:[]]
[M:3 W/V:20.5/25 U:[]]
[M:8 W/V:3.5/9 U:[]]
[M:1 W/V:6.5/12 U:[]]
[M:2 W/

[M:8 W/V:16.5/79 U:[]]
[M:3 W/V:53.0/158 U:[]]
[M:5 W/V:18.0/83 U:[]]
[M:0 W/V:58.0/170 U:[]]
[M:6 W/V:43.5/140 U:[]]
[M:4 W/V:59.0/172 U:[]]
[M:1 W/V:32.0/115 U:[]]
[M:7 W/V:18.0/83 U:[]]

Best Move: 4

..X
.O.
...

[M:5 W/V:4.5/10 U:[]]
[M:6 W/V:5.5/11 U:[]]
[M:7 W/V:5.5/11 U:[]]
[M:1 W/V:18.0/23 U:[]]
[M:8 W/V:6.5/12 U:[]]
[M:0 W/V:13.5/19 U:[]]
[M:3 W/V:7.5/14 U:[]]

Best Move: 1

.XX
.O.
...

[M:7 W/V:4.5/23 U:[]]
[M:0 W/V:753.0/867 U:[]]
[M:8 W/V:8.0/28 U:[]]
[M:5 W/V:8.5/29 U:[]]
[M:3 W/V:8.0/28 U:[]]
[M:6 W/V:6.0/25 U:[]]

Best Move: 0

OXX
.O.
...

[M:5 W/V:5.0/20 U:[]]
[M:6 W/V:6.5/21 U:[]]
[M:7 W/V:0.5/12 U:[]]
[M:3 W/V:3.5/18 U:[]]
[M:8 W/V:10.5/29 U:[]]

Best Move: 8

OXX
.O.
..X

[M:5 W/V:41.5/65 U:[]]
[M:7 W/V:2.5/16 U:[]]
[M:6 W/V:22.5/43 U:[]]
[M:3 W/V:857.0/876 U:[]]

Best Move: 3

OXX
OO.
..X

[M:5 W/V:5.0/39 U:[]]
[M:7 W/V:0.0/25 U:[]]
[M:6 W/V:4.0/36 U:[]]

Best Move: 5

OXX
OOX
..X

P1 all states: 
 [[1 0 0 1 0 0 0 0 0 0 4]
 [1 0 1 1 0 2 0 0 0 0 0]
 [1 2 1 1 0 2 0


Best Move: 3

X..
OO.
X..

[M:1 W/V:13.5/23 U:[]]
[M:7 W/V:6.0/15 U:[]]
[M:2 W/V:11.5/21 U:[]]
[M:5 W/V:12.0/21 U:[]]
[M:8 W/V:10.5/20 U:[]]

Best Move: 1

XX.
OO.
X..

[M:8 W/V:7.5/51 U:[]]
[M:7 W/V:7.0/50 U:[]]
[M:5 W/V:10.0/57 U:[]]
[M:2 W/V:459.0/842 U:[]]

Best Move: 2

XXO
OO.
X..

[M:8 W/V:3.0/19 U:[]]
[M:5 W/V:32.0/64 U:[]]
[M:7 W/V:2.0/17 U:[]]

Best Move: 5

XXO
OOX
X..

[M:8 W/V:250.0/500 U:[]]
[M:7 W/V:250.0/500 U:[]]

Best Move: 7

XXO
OOX
XO.

[M:8 W/V:50.0/100 U:[]]

Best Move: 8

XXO
OOX
XOX

P1 all states: 
 [[1 1 0 0 0 0 0 0 0 0 4]
 [1 1 0 0 0 2 0 1 0 0 3]
 [1 1 1 0 2 2 0 1 0 0 2]
 [1 1 1 2 2 2 1 1 0 0 7]] 

P2 all states: 
 [[2 0 0 0 0 0 0 0 0 0 0]
 [2 1 0 0 0 2 0 0 0 0 6]
 [2 1 0 0 2 2 0 1 0 0 1]
 [2 1 1 2 2 2 0 1 0 0 5]
 [2 1 1 2 2 2 1 1 2 0 8]] 

Nobody wins!
...
...
...

[M:2 W/V:8.5/11 U:[]]
[M:0 W/V:13.5/15 U:[]]
[M:4 W/V:13.0/15 U:[]]
[M:3 W/V:8.5/11 U:[]]
[M:8 W/V:8.5/11 U:[]]
[M:5 W/V:2.0/6 U:[0, 1, 3]]
[M:6 W/V:9.5/12 U:[]]
[M:1 W/V:4.5/8 U:[2]]
[M:7 W/V:8